In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("../input/vehicle-dataset-from-cardekho/car data.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
print(df['Seller_Type'].unique())
print(df['Owner'].unique())
print(df['Transmission'].unique())
print(df['Fuel_Type'].unique())

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

In [ ]:
final_dataset = df[['Year', 'Selling_Price', 'Present_Price', 'Kms_Driven',
       'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner']]

In [ ]:
final_dataset.head()

In [ ]:
final_dataset['current_year'] =2020

In [ ]:
final_dataset.head()

In [ ]:
final_dataset['year_old'] = final_dataset['current_year'] - final_dataset['Year']

In [ ]:
final_dataset.head()

In [ ]:
final_dataset.drop(['Year', 'current_year'], axis=1, inplace=True)

In [ ]:
final_dataset.head()

In [ ]:
final_dataset_dummy = pd.get_dummies(final_dataset, drop_first=True)

In [ ]:
final_dataset_dummy.head()

In [ ]:
final_dataset_dummy.corr()

In [ ]:
import seaborn as sns
sns.pairplot(final_dataset_dummy)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
corrmat = final_dataset_dummy.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
g=sns.heatmap(final_dataset_dummy[top_corr_features].corr(), annot=True, cmap="RdYlGn")

In [ ]:
X = final_dataset_dummy.iloc[:,1:]
y = final_dataset_dummy.iloc[:,0]

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()
model.fit(X,y)

In [ ]:
print(model.feature_importances_)

In [ ]:
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(5).plot(kind='barh')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_train.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

In [ ]:
import numpy as np

In [ ]:
n_estimators = [int(x) for x in np.linspace(100,1200,12)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(5,30,6)]
min_samples_split = [2,5,10,15,100]
min_samples_leaf = [1,2,5,10]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

In [ ]:
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, scoring='neg_mean_squared_error', n_iter=10, cv=5, verbose=2, random_state=42, n_jobs=-1)

In [ ]:
rf_random.fit(X_train, y_train)

In [ ]:
rf_random.best_params_

In [ ]:
predictions = rf_random.predict(X_test)
predictions

In [ ]:
sns.distplot(y_test-predictions)

In [ ]:
sns.scatterplot(y_test, predictions)

In [ ]:
import pickle
pickle.dump(rf_random, open('random_forest_regression_model.pkl', 'wb'))